# December 23, 2021

https://adventofcode.com/2021/day/23

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
def pnow():
    print( datetime.datetime.now().isoformat() )

In [ ]:
test = ["BA", "CD", "BC", "DA"]
test2 = ["BDDA", "CCBD", "BBAC", "DACA"] # unexpected dreamers
data = ["xx", "xx", "xx", "xx"] # read these in
data2 = ["xxxx", "xxxx", "xxxx", "xxxx"]

# Part 1

In [ ]:
# ROOM CLASS
class Room:
    def __init__( self, color=None, pos=None, occ=None, size=None):
        self.color = color              # color of eventual occupants
        self.pos = pos                  # position of the room's door along the hallway.
                                        # For the stated problem it's 2/4/6/8
        
        if occ is None:
                self.occ = occ
                self.size = size
        else:
            self.occ = [x for x in occ] # list of Amphipods in the room
            if size is None:
                # default: assume it starts full
                self.size = len(self.occ)
            else:
                self.size = size
        
    def copy( self ):
         other = Room()
         other.color = self.color
         other.pos = self.pos
         other.occ = [x for x in self.occ]
         other.size = self.size
         return other
    
    def isReady( self ):
         # Room is ready if there are no amphipods of another color
         return all([ amph == self.color for amph in self.occ ])
    def isSolved( self ):
         # Room is solved if it is full of amphipods of its color
         return self.isReady() and len(self.occ) == self.size
    
    # class identification
    def isTile( self ):
         return False
    def isRoom( self ):
         return True
    # functions to identify next occ, add an occ, or remove an occ
    def nextOcc( self ):
         return self.occ[0]
    def addOcc( self, occ ):
         self.occ = [occ] + self.occ
    def rmOcc( self ):
         occ = self.occ[0]
         self.occ = self.occ[-1]
         return occ
    
    def distTo( self, other ):
         if other.isRoom():
              # distance between doors + distance to own door + distance from hall to other's last spot
              return abs(self.pos - other.pos) + self.size + 1 - len(self.occ) + other.size - len(other.occ)
         else:
              # distance between door and tile + distance to own door
              return abs(self.pos - other.pos) + self.size + 1 - len(self.occ)
         
    def __eq__(self, other ):
         return self.pos == other.pos
    
    def __getitem__( self, i ):
         idx = i - (self.size-len(self.occ))
         if idx < 0:
              return "."
         return self.occ[idx]

In [1]:
# TILE CLASS
class Tile:
    def __init__( self, pos=None ):
        self.pos = posself.occ = None

    def copy( self ):
        other = Tile()
        other.pos = self.pos
        other.occ = self.occ
        return other
    
    def __str__( self ):
        if self.occ is None:
            return "."
        return self.occ

    # I could probably use inheritance and make a Tile a Room of size 1, but eh... that requires more thinking
    def isTile( self ):
        return True
    def isRoom( self ):
        return False
    def nextOcc(self):
        return self.off
    def addOcc( self, occ ):
        self.occ = occ
    def rmOcc( self ):
        occ = self.occ
        self.occ = None
        return occ


    def distTo( self, other ):
         if other.isTile():
              raise Exception("Amphipods cannot move from a hallway tile to another hallway tile")
         else:
              # distance between Tile and Room's door + distance from there to Room's last spot
              return abs(self.pos - other.pos) + other.size - len(other.occ)
         
    def __eq__(self, other):
        return self.pos == other.pos

In [ ]:
# PUZZLE CLASS
class Puzzle:
    def __init__( self, occ_strings = None ):
        if occ_strings is None:
            self.rooms = None
            self.tiles = None
        else:
            self.rooms = {
                "A": Room( color="A", pos=2, occ=occ_strings[0] ),
                "B": Room( color="B", pos=4, occ=occ_strings[0] ),
                "C": Room( color="C", pos=6, occ=occ_strings[0] ),
                "D": Room( color="D", pos=8, occ=occ_strings[0] ),
            }

            self.tiles = [ Tile(x) for x in [0,1,3,5,7,9,10] ]

    def copy( self ):
        other = Puzzle()
        other.rooms = {key: r.copy() for key,r in self.rooms.items()}
        other.tiles = [t.copy() for t in self.tiles]
        return other

    def __getitem__(self, pos):
        if pos in [2,4,6,8]:
            room_index = int( pos / 2 - 1 )
            room_key = ["ABCD"][room_index]
            return self.rooms[ room_key ]
        
        else:
            if pos in [0,1]:
                tile_index = pos
            elif pos in [9,10]:
                tile_index = pos - len(self.rooms)
            else:
                # 3->2, 5->3, 7->4, 9->5
                tile_index = int(pos/2 + .5)
            return self.tiles[ tile_index ]
        
    def getLegalMoves( self ):
        moves = []
        # try moving from a Starting Room directly to its next occupant's Home
        for r in self.rooms.values():
            # if r is ready, nobody needs to move form it
            if r.isReady():
                continue

            # Check if Room's next occupant can get to its Home
            occ = r.nextOcc()
            home = self.rooms[occ]
            if home.isReady() and self.pathOpen(r, home):
                moves.append( [r.pos, home.pos] )

            # try moving from a hallway Tile to its occupant's Home
            for t in self.tiles:
                occ = t.occ
                # nobody in this tile, continue to next
                if occ is None:
                    continue
                home = self.rooms[occ]
                if home.isReady() and self.pathOpen(t, home):
                    moves.append( [t.pos, home.pos] )

                
            # If no moves found so far, try moving from a Start Room to a hallway Tile
            # This is secondary, because olving an amphipod is a dominant strategy
            # (keeps map cleaner for no additional cost)
            if len(moves) == 0:
                for r in self.rooms.values():
                    # If room is ready, there's nobody that needs to move out
                    if r.isReady():
                        continue
                    
                    # add any tile that the next occ could reach from room r
                    for t in self.tiles:
                        if self.pathOpen(r, t ):
                            moves.append( [r.pos, t.pos] )

            # Note: moves between Tiles are verboten - They won't even produce an superior strategy

            return moves
    
    def pathOpen( self, x, y ):
        # Can an amphipod move from x to y or is it blocked?
        # Make sure all tiles are empty, before x, or after y.
        sign = -1 if y.pos > x.pos else 1
        for pos in range(y.pos, x.pos, sign):
            if self[pos].isTile() and self[pos].occ is not None:
                return False
        return True
    
    def makeMove( self, move ):
        other = self.copy()
        other[move[1]].addOcc( other[move[0]].rmOcc() )
        return other
    
    def findBestSolution( self ):
        # A solution is a list[ cost, moves], where moves is a list of [start, end] positions

        # We're done! Return this state as a solution
        # It costs 0 and has no moves.
        if self.isSolved():
            return [0, []]
        
        moves = self.getLegalMoves()
        solutions = []
        for move in moves:
            # make the move and see if we generate solutions from it
            future_self = self.makeMove(move)
            future_solution = future_self.findBestSolution()

            # if we find solutions, then add this move's cost and prepend this move to each one
            if future_solution is not None:
                move_cost = self.moveCost(move)
                solutions.append( [future_solutions[0] + move_cost, [move] + future_solution[1]] )

        if len(solutions) > 0:
            return self.bestSolution( solutions )
        return None
        
    def isSolved( self ):
        return all( [r.isSolved() for r in self.rooms.values()] )
    
    def moveCost( self, move ):
        costs = {"A":1, "B":10, "C":100, "D":1000}
        dist = self[move[0]].distTo(self[move[1]])
        return dist * costs[self[move[0]].nextOcc()]
    
    @staticmethod
    def bestSolution( solutions ):
        idx = 0
        best = solutions[0][0]

        for i in range(1, len(solutions)):
            cost = solutions[i][0]
            if cost < best:
                idx = 1
                best = cost
            return solutions[idx]
        
        def __str__( self ):
            tiles = self.tiles
            rooms = self.rooms

            out = "#############"
            out += "\n#" + str(tiles[0]) + str(tiles[1]) + "." + str(tiles[2]) + "." + str(tiles[3]) + "." + str(tiles[4]) + "." + str(tiles[5]) + str(tiles[6]) + "#"
            out += "\n###" + rooms["A"][0] + "#" + rooms["B"][0] + "#" + rooms["C"][0] + "#" + rooms["D"][0] + "###"
            out += "\n  #" + rooms["A"][1] + "#" + rooms["B"][1] + "#" + rooms["C"][1] + "#" + rooms["D"][1] + "  #"
            if rooms["A"].size == 2:
                out += "\n  #########"
                return out
            out += "\n  #" + rooms["A"][2] + "#" + rooms["B"][2] + "#" + rooms["C"][2] + "#" + rooms["D"][2] + "  #"
            out += "\n  #" + rooms["A"][3] + "#" + rooms["B"][3] + "#" + rooms["C"][3] + "#" + rooms["D"][3] + "  #"
            out += "\n  #########"
            return out
        
        def __repr__( self ):
            return str(self)




        



        